Proposition 1:

- Sales Order Ranking by Cutomers.

11091 63636 28.1554 7 7

11091 73198 30.1444 8 8

11091 73594 30.1444 8 8

11091 55626 30.6859 10 9

11091 74480 32.0229 11 10

In [ ]:
USE AdventureWorks2017

SELECT CustomerID,
SalesOrderID,
TotalDue,
RANK() OVER (PARTITION BY CustomerID ORDER BY TotalDue ASC) AS Rank,
DENSE_RANK() OVER (PARTITION BY CustomerID ORDER BY TotalDue ASC) AS DenseRank

FROM Sales.SalesOrderHeader
ORDER BY CustomerID, TotalDue


Proposition 2: 

- Distinct Tax Amounts with Row Number:

CTE, Row\_Number(), Orderby Inside, Order by outside.

  

Useful for retrieving unique values from a column and assigning a sequestal identifier.

In [ ]:

USE AdventureWorks2017;

WITH DistinctTax AS (
    SELECT DISTINCT TaxAmt
    FROM Sales.SalesOrderHeader
)
SELECT 
    TaxAmt,
    ROW_NUMBER() OVER (ORDER BY TaxAmt) AS RowNum
FROM 
    DistinctTax
ORDER BY 
    TaxAmt;



Proposition 3:

- Find year over year changes in total ordervalue for each customer

1. CustomerYearlySales CTEFind the total ordervalue for each year by joining sod table and soh. Then find the sum for each year.
2. Lag() used to find the previous year value for each customer.
3. Partition by customerID to ensure the lag() and lead() operate seperately for each customerOrder by orderyear ensures that calculations are based on order year.

Use case to track consistent customers.

In [ ]:
USE AdventureWorks2017;

WITH CustomerYearlySales AS (
    SELECT 
        soh.CustomerID,
        YEAR(soh.OrderDate) AS OrderYear,
        SUM(sod.LineTotal) AS TotalOrderValue
    FROM 
        Sales.SalesOrderDetail AS sod
    JOIN 
        Sales.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
    GROUP BY 
        soh.CustomerID, YEAR(soh.OrderDate)
)

SELECT 
    cys.CustomerID,
    cys.OrderYear,
    cys.TotalOrderValue,
    LAG(cys.TotalOrderValue) OVER (PARTITION BY cys.CustomerID ORDER BY cys.OrderYear) AS PrevYearOrderValue,
    LEAD(cys.TotalOrderValue) OVER (PARTITION BY cys.CustomerID ORDER BY cys.OrderYear) AS NextYearOrderValue

FROM 
    CustomerYearlySales AS cys
ORDER BY 
    cys.CustomerID, cys.OrderYear;


Proposition 4:

- Calculate the difference in quantity between consecutive sales for each product.

In [ ]:
USE AdventureWorks2017;
SELECT ProductID, SalesOrderID, OrderQty,
  OrderQty - LAG(OrderQty) OVER(PARTITION BY ProductID ORDER BY SalesOrderID) AS QtyDifference
FROM Sales.SalesOrderDetail
ORDER BY ProductID, SalesOrderID;

Proposition 5:

- Find the Products with the highest and lowest sales in each year.

In [ ]:
USE AdventureWorks2017;

SELECT YEAR(SO.OrderDate) AS SalesYear, ProductID, SUM(SOD.OrderQty) AS TotalSales,
  RANK() OVER(PARTITION BY YEAR(SO.OrderDate) ORDER BY SUM(SOD.OrderQty) DESC) AS HighestSalesRank
FROM Sales.SalesOrderDetail SOD
JOIN Sales.SalesOrderHeader SO ON SOD.SalesOrderID = SO.SalesOrderID
GROUP BY ProductID, YEAR(SO.OrderDate)
ORDER BY SalesYear, HighestSalesRank;




Proposition 6:

- Yearly sales revenue per territoty

In [ ]:
USE AdventureWorks2017;


SELECT TerritoryID, [2012] AS Revenue2012, [2013] AS Revenue2013, [2014] AS Revenue2014
FROM (
    SELECT TerritoryID, YEAR(OrderDate) AS SalesYear, TotalDue
    FROM Sales.SalesOrderHeader
) AS src
PIVOT(
    SUM(TotalDue)
    FOR SalesYear IN ([2012], [2013], [2014])
) AS pvt
Order BY TerritoryID


Proposition 7:

- Employee sales per year

In [ ]:
USE AdventureWorks2017
SELECT ProductID, Month, TotalSales
FROM (
    SELECT ProductID, 
           SUM(CASE WHEN MONTH(OrderDate) = 1 THEN LineTotal ELSE 0 END) AS Jan,
           SUM(CASE WHEN MONTH(OrderDate) = 2 THEN LineTotal ELSE 0 END) AS Feb,
           SUM(CASE WHEN MONTH(OrderDate) = 3 THEN LineTotal ELSE 0 END) AS Mar
           
    FROM Sales.SalesOrderDetail
    JOIN Sales.SalesOrderHeader ON SalesOrderDetail.SalesOrderID = SalesOrderHeader.SalesOrderID
    GROUP BY ProductID
) AS SalesData
UNPIVOT(
    TotalSales FOR Month IN (Jan, Feb, Mar)
) AS Unpvt;


Proposition 8: 

- Find the top 3 recent orders for each customer.

In [ ]:
USE AdventureWorks2017
SELECT CustomerID, SalesOrderID, OrderDate, TotalDue
FROM Sales.Customer AS c
CROSS APPLY (
    SELECT TOP 3 SalesOrderID, OrderDate, TotalDue
    FROM Sales.SalesOrderHeader AS soh
    WHERE soh.CustomerID = c.CustomerID
    ORDER BY OrderDate DESC
) AS RecentOrders;


Proposition 9:

- Calculate total and average line tiem cost for each product.

In [ ]:
use AdventureWorks2017
SELECT ProductID, TotalLineCost, AverageLineCost
FROM Production.Product AS p
CROSS APPLY (
    SELECT SUM(LineTotal) AS TotalLineCost, AVG(LineTotal) AS AverageLineCost
    FROM Sales.SalesOrderDetail AS sod
    WHERE sod.ProductID = p.ProductID
) AS LineItemCosts
where LineItemCosts.TotalLineCost is not NULL 
ORDER BY ProductID


Proposition 10:

- Calculating the differene in linetotal for consecutive sales order line by product

In [58]:

USE AdventureWorks2017;

SELECT ProductID, SalesOrderID, LineTotal,
  LineTotal - LAG(LineTotal) OVER(PARTITION BY ProductID ORDER BY SalesOrderID) AS LineTotalDifference
FROM Sales.SalesOrderDetail
ORDER BY ProductID, SalesOrderID;





(121317 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:01.260

ProductID,SalesOrderID,LineTotal,LineTotalDifference
707,43665,20.186500,NULL
707,43668,40.373000,20.186500
707,43673,80.746000,40.373000
707,43677,20.186500,-60.559500
707,43678,20.186500,0.000000
707,43680,80.746000,60.559500
707,43681,20.186500,-60.559500
707,43683,60.559500,40.373000
707,43692,80.746000,20.186500
707,43693,40.373000,-40.373000
